In [ ]:
import pickle

def analyze(path):
    with open("{}.pkl".format(path), "rb+") as file:
        data = pickle.load(file)

    # resort
    for seed, epi_data in data.items():
        for obs_index, obs_ret in epi_data.items():
            if obs_index == "seed":
                continue
            resort = sorted(obs_ret, key=lambda x: x["neuron"]["neuron_index"], reverse=True)
            resort = sorted(resort, key=lambda x: x["neuron"]["layer"], reverse=True)
            epi_data[obs_index] = resort

            # average
    averaged_ret = {}
    for seed, epi_data in data.items():
        for obs, obs_ret in epi_data.items():
            if obs == "seed":
                continue
            if obs in averaged_ret:
                for k,each_neuron in enumerate(obs_ret):
                    assert averaged_ret[obs][k]["layer"] == each_neuron["neuron"]["layer"] and averaged_ret[obs][k]["neuron_index"] == each_neuron["neuron"]["neuron_index"], "{} {}".format(averaged_ret[obs][k]["neuron_index"], each_neuron["neuron"]["neuron_index"])
                    averaged_ret[obs][k]["freq_diff"] = (averaged_ret[obs][k]["freq_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["freq_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                    averaged_ret[obs][k]["correlation"] = (averaged_ret[obs][k]["correlation"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["correlation"])/(averaged_ret[obs][k]["seed_num"]+1)
                    averaged_ret[obs][k]["seed_num"]+=1
            else:
                averaged_ret[obs] = []
                for each_neuron in obs_ret:
                    averaged_ret[obs].append({"layer": each_neuron["neuron"]["layer"],
                                       "neuron_index": each_neuron["neuron"]["neuron_index"],
                                       "freq_diff": each_neuron["error"]["freq_diff"],
                                       "correlation": each_neuron["error"]["correlation"],
                                       "seed_num": 1})
    for obs_dim, avg_ret in averaged_ret.items():
        averaged_ret[obs_dim] = sorted(avg_ret, key=lambda x:x["freq_diff"],reverse=False)
    return averaged_ret

In [ ]:
# For anymal:
averaged_ret = analyze("anymal_ret")

In [ ]:
# 5: angular velocity (yaw)
print("angular velocity related neuron:", averaged_ret[5][1]) # 1 better than 0 due to a higher correlation
# 1: x-linear velocity
print("x-linear velocity related neuron:", averaged_ret[1][1]) # 1 better than 0 due to a higher correlation

In [64]:
# For MetaDrive:
averaged_ret = analyze("metadrive_ret")

In [67]:
print("side distance related neuron:", averaged_ret[0][0]) 
print("speed related neuron:", averaged_ret[3][0])

side distance related neuron: {'layer': 0, 'neuron_index': 1, 'freq_diff': 10.60877, 'correlation': 1.0, 'seed_num': 1}
speed related neuron: {'layer': 0, 'neuron_index': 249, 'freq_diff': 12.7400255, 'correlation': -1.000000207437052, 'seed_num': 1}
